# Neural Matrix Factorization (NeuMF)

In [33]:
import numpy as np
import math
import tensorflow as tf
import scipy.sparse as sp
import heapq

from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential, Model

from tensorflow.keras.layers import Embedding, Input, Dense, Reshape, Flatten, Dropout
from tensorflow.keras.layers import Concatenate, Multiply
from tensorflow.keras.optimizers import Adam

from tensorflow.keras import initializers

from tensorflow.keras.metrics import MeanSquaredError, Precision, AUC

## Import Train and Test Data

In [2]:
train_filename = "small_train.csv"

num_users, num_articles = 0, 0
with open(train_filename, "r") as f:
    header = f.readline()
    line = f.readline()
    while line != None and line != "":
        line_list = line.split(",")
        u, i = int(line_list[2]), int(line_list[3])
        num_users = max(num_users, u)
        num_articles = max(num_articles, i)
        line = f.readline()

num_users += 1
num_articles += 1

In [3]:
num_users, num_articles

(40331, 31415)

In [4]:
train = sp.dok_matrix((num_users, num_articles), dtype=np.float32)

with open(train_filename, "r") as f:
    header = f.readline()
    print(header)
    line = f.readline()
    while line != None and line != "":
        line_list = line.split(",")
        user, article = int(line_list[2]), int(line_list[3])
        train[user, article] = 1.0
        line = f.readline()

user_id,article,user_id_code,article_id_code



In [5]:
test_filename = "small_test.csv"

test_positives = []
with open(test_filename, "r") as f:
    header = f.readline()
    print(header)
    line = f.readline()
    print(line)
    while line != None and line != "":
        line_list = line.split(",")
        #print(line_list)
        user, article = int(line_list[2]), int(line_list[3])
        #print(user, article)                                            
        test_positives.append([user, article])
        line = f.readline()

user_id,article,user_id_code,article_id_code

U13740,N31801,1810,11677



In [6]:
len(test_positives)

39846

In [7]:
test_neg_filename = "small_test_negatives.tsv"

test_negatives = []
with open(test_neg_filename, "r") as f:
    line = f.readline()
    while line != None and line != "":
        line_list = line.split("\t")
        #print(line_list)
        negatives = []
        for neg in line_list[1: ]:
            negatives.append(int(neg))
        test_negatives.append(negatives)
        line = f.readline()

In [8]:
len(test_negatives)

39846

## Initialize Model Parameters

In [9]:
EPOCHS = 20
BATCH_SIZE = 256
NUM_FACTORS = 8
LAYERS = [64,32,16,8]
REG_MF = 0
REG_LAYERS = [0,0,0,0]
REGS = [0, 0]
NUM_NEG = 4
LR = 0.001
LEARNER = "adam"

In [10]:
topK = 10

In [11]:
NUM_LAYER = len(LAYERS)

## Build Model

In [36]:
user_input = Input(shape=(1,), dtype='int32', name='user_input')
article_input = Input(shape=(1,), dtype='int32', name='article_input')

### User and Article Embeddings

#### Matrix Factorization

In [37]:
MF_Embedding_User = Embedding(input_dim=num_users, 
                              output_dim=NUM_FACTORS, 
                              name='mf_user_embedding',
                              input_length=1)

In [38]:
MF_Embedding_Article = Embedding(input_dim=num_articles, 
                                 output_dim=NUM_FACTORS, 
                                 name = 'mf_article_embedding',
                                 input_length=1)

#### Mulit-Layer Perceptron

In [39]:
MLP_Embedding_User = Embedding(input_dim=num_users, output_dim=LAYERS[0]//2, 
                               name='mlp_user_embedding', input_length=1)

In [40]:
MLP_Embedding_Article = Embedding(input_dim=num_articles, output_dim=LAYERS[0]//2, 
                               name='mlp_article_embedding', input_length=1)

### MF and MLP Prediction

In [41]:
mf_user_latent = Flatten()(MF_Embedding_User(user_input))
mf_article_latent = Flatten()(MF_Embedding_Article(article_input))

mf_vector = Multiply()([mf_user_latent, mf_article_latent])

In [42]:
mlp_user_latent = Flatten()(MLP_Embedding_User(user_input))
mlp_article_latent = Flatten()(MLP_Embedding_Article(article_input))

mlp_vector = Concatenate(axis=-1)([mlp_user_latent, mlp_article_latent])

In [43]:
for idx in range(1, NUM_LAYER):
    layer = Dense(LAYERS[idx], activation='relu', name='layer%d' %idx)
    mlp_vector = layer(mlp_vector)

In [44]:
predict_vector = Concatenate()([mf_vector, mlp_vector])

In [45]:
prediction = Dense(1, activation='sigmoid', 
                   kernel_initializer='lecun_uniform', 
                   name = 'prediction')(predict_vector)

## Compile and Fit Model

In [46]:
model = Model([user_input, article_input], prediction)

In [47]:
model.compile(optimizer=Adam(lr=LR), loss='binary_crossentropy',
             metrics=[MeanSquaredError(), Precision(), AUC()])

In [48]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
article_input (InputLayer)      [(None, 1)]          0                                            
__________________________________________________________________________________________________
mlp_user_embedding (Embedding)  (None, 1, 32)        1290592     user_input[0][0]                 
__________________________________________________________________________________________________
mlp_article_embedding (Embeddin (None, 1, 32)        1005280     article_input[0][0]              
____________________________________________________________________________________________

In [49]:
user_input, article_input, labels = [],[],[]
for (u, i) in train.keys():
    # positive instance
    user_input.append(u)
    article_input.append(i)
    labels.append(1)
    # negative instances
    for t in range(NUM_NEG):
        j = np.random.randint(num_articles)
        while (u, j) in train.keys():
            j = np.random.randint(num_articles)
        user_input.append(u)
        article_input.append(j)
        labels.append(0)

In [55]:
from sklearn.model_selection import train_test_split

In [71]:
zipped = list(zip(user_input, article_input, labels))

In [72]:
zipped[0]

(1810, 24230, 1)

In [73]:
train, test = train_test_split(zipped, random_state=420)

In [74]:
train_users = set([ele[0] for ele in train])
train_items = set([ele[1] for ele in train])

test_users = set([ele[0] for ele in test])
test_items = set([ele[1] for ele in test])

In [75]:
len(test_users - train_users)

0

In [76]:
len(test_items - train_items)

0

In [50]:
hist = model.fit([np.array(user_input), np.array(article_input)], #input
                 np.array(labels), # labels 
                 batch_size=BATCH_SIZE, 
                 epochs=1, 
                 verbose=1, 
                 shuffle=True)

4042170/4042170 [==============================] - 294s 73us/sample - loss: 0.2902 - mean_squared_error: 0.0887 - precision_1: 0.7407 - auc_1: 0.9069


## Evaluate Model

In [51]:
def eval_one_rating(idx, topK):
    rating = test_positives[idx]
    items = test_negatives[idx]
    u = rating[0]
    get_item = rating[1]
    items.append(get_item)
    # Get prediction scores
    map_item_score = {}
    users = np.full(len(items), u, dtype = 'int32')
    predictions = model.predict([users, np.array(items)], 
                                batch_size=100, verbose=0)
    for i in range(len(items)):
        item = items[i]
        map_item_score[item] = predictions[i]
    items.pop()
    
    # Evaluate top rank list
    ranklist = heapq.nlargest(topK, map_item_score, key=map_item_score.get)
    
    if get_item in ranklist:
        hr = 1
        i = ranklist.index(get_item)
        ndcg = math.log(2) / math.log(i+2)
        rr = 1/(i+1)
    else:
        hr = 0
        ndcg = 0
        rr = 0
   
    return (hr, ndcg, rr)

In [52]:
hits, ndcgs, rrs = [], [], []
for idx in range(len(test_positives)):
    hr, ndcg, rr = eval_one_rating(idx, topK)
    hits.append(hr)
    ndcgs.append(ndcg)
    rrs.append(rr)

In [53]:
hr = np.array(hits).mean()
mrr = np.array(rrs).mean()
ndcg = np.array(ndcgs).mean()

print("Hit ratio:            ", hr)
print("Mean reciprocal rank: ", mrr)
print(f"NDCG@{topK}:         ", ndcg)

Hit ratio:             0.7604025498167947
Mean reciprocal rank:  0.39368861784537135
NDCG@10:               0.48043475817742387
